# SD202 TP3 - Views, Updates and Database Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network


For the first 2 parts we will use the _wine_ database and the Tables created in TP2.

A reminder of the wine database schema:

In [2]:
import sqlite3

In [3]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [4]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
CLIENT:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
GEOGRAPHY:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
TRANSACTIONS:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
WINE:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we recommend inline %sql as an alternative to the sqlite3 package

In [5]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: None@wine.db'

Recreate the Tables in 3NF/BCNF from Master2 as you did in the TP2

In [6]:
%sql DROP TABLE IF EXISTS WINE;
%sql DROP TABLE IF EXISTS CLIENT;
%sql DROP TABLE IF EXISTS TRANSACTIONS;

# Create WINE : We concatenate with the precedent one from MASTER1
%sql CREATE TABLE WINE AS \
SELECT DISTINCT NV, CRU, DEGRE, MILL \
FROM MASTER2 \
WHERE NV IS NOT null \
UNION \
SELECT DISTINCT NV, CRU, DEGRE, MILL \
FROM MASTER1 \
WHERE NV IS NOT null;

# Create CLIENT
%sql CREATE TABLE CLIENT AS \
SELECT DISTINCT NB, NOM, PRENOM, TYPE \
FROM MASTER2 \
WHERE NB IS NOT null;

# Create TRANSACTIONS
%sql CREATE TABLE TRANSACTIONS AS \
SELECT DISTINCT NB, NV, DATES, LIEU, QTE \
FROM MASTER2 \
WHERE NV IS NOT NULL \
AND NB IS NOT NULL;

# Create GEOGRAPHY
%sql CREATE TABLE GEOGRAPHY AS \
SELECT DISTINCT LIEU, REGION \
FROM MASTER2 \
WHERE LIEU IS NOT NULL;
print("\nContent of the database")
printSchema(conn)

Done.
Done.
Done.
Done.
Done.
Done.
(sqlite3.OperationalError) table GEOGRAPHY already exists [SQL: 'CREATE TABLE GEOGRAPHY AS SELECT DISTINCT LIEU, REGION FROM MASTER2 WHERE LIEU IS NOT NULL;']

Content of the database
CLIENT:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
GEOGRAPHY:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
TRANSACTIONS:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
WINE:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```sql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command is:

```sql
DROP VIEW IF EXISTS view_name;
```


__Note:__ Use it with caution (only drop something if you are sure)

__1.1__ Create a view 'bons_buveurs' with the clients (buveurs) of type 'gros' or 'moyen'.

In [5]:
%%sql 

DROP VIEW IF EXISTS bons_buveurs;

CREATE VIEW bons_buveurs AS
SELECT *
FROM CLIENT
WHERE TYPE in ('gros', 'moyen');

Done.
Done.


[]

In [6]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


__1.2__ Create the view 'buveurs_asec' with clients (buveurs) who have not bought any wine.

In [7]:
%%sql 

DROP VIEW IF EXISTS buveurs_asec;

CREATE VIEW buveurs_asec AS
SELECT *
FROM CLIENT
WHERE CLIENT.NB NOT IN (SELECT TRANSACTIONS.NB
                        FROM TRANSACTIONS)

Done.
Done.


[]

In [8]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__1.3__ Create the view 'buveurs_achats' complementary to the previous one.

In [9]:
%%sql 

DROP VIEW IF EXISTS buveurs_achats;

CREATE VIEW buveurs_achats AS
SELECT *
FROM CLIENT
WHERE CLIENT.NB IN (SELECT TRANSACTIONS.NB
                    FROM TRANSACTIONS)

Done.
Done.


[]

In [10]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


__1.4__ Create the view 'q83pl' (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [11]:
%%sql 

DROP VIEW IF EXISTS q83pl;

CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE, DATES
FROM TRANSACTIONS, WINE
WHERE DATES LIKE '1983-%'
AND TRANSACTIONS.NV = WINE.NV
GROUP BY LIEU, CRU

Done.
Done.


[]

In [12]:
# Test
%sql SELECT * FROM q83pl;

Done.


LIEU,CRU,QTE_BUE,DATES
CAEN,Seyssel,3,1983-02-21
LILLE,Pommard,5,1983-06-29
LYON,Beaujolais Villages,10,1983-06-06
LYON,Julienas,2,1983-12-25
PARIS,Beaujolais Primeur,4,1983-03-10
PARIS,Coteaux du Tricastin,1,1983-12-31
PARIS,Pouilly Vinzelles,3,1983-02-14
RENNES,Mercurey,1,1983-12-31
ROCQUENCOURT,Beaujolais Villages,260,1983-05-05
ROCQUENCOURT,Saint Amour,80,1983-05-05


__1.5__ Can we define the same view with ascending order over the attribute QTE? Provide an explanation for your answer.

__
# PART II: UPDATE DATABASE CONTENT

The syntax for the Update statement is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the Insert statement is:

```sql
INSERT INTO TABLE_NAME [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```


Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you may not execute a DELETE, INSERT or UPDATE statement on a view.

__2.1__ Create a table 'RBB' with the same schema as 'bons_buveurs' which contains the tuples selected from 'bons_buveurs'

In [13]:
%%sql 
DROP TABLE IF EXISTS RBB;

CREATE TABLE RBB AS 
SELECT * 
FROM bons_buveurs
ORDER BY NB;

Done.
Done.


[]

In [14]:
# Test
%sql SELECT * FROM RBB;

Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


__2.2__ Update the table you used to create 'bons_buveurs': Change the 'type' to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may be different from the one in your table)

In [15]:
%%sql
SELECT CLIENT.NB AS NB, PRENOM, NOM, TYPE, SUM(QTE) AS TOTAL
FROM CLIENT, TRANSACTIONS
WHERE CLIENT.NB = TRANSACTIONS.NB
AND TYPE != 'gros'
GROUP BY CLIENT.NB
HAVING TOTAL > 100

Done.


NB,PRENOM,NOM,TYPE,TOTAL
2,Antonin,Artaud,moyen,583
5,Jacques,Audiberti,petit,113
9,Emile,Ajar,petit,140
44,Andre,Gide,petit,171


Update instances

In [16]:
%%sql

UPDATE CLIENT
SET TYPE = 'gros'
WHERE NB IN (SELECT CLIENT.NB
            FROM CLIENT, TRANSACTIONS
            WHERE CLIENT.NB = TRANSACTIONS.NB
            AND TYPE != 'gros'
            GROUP BY CLIENT.NB
            HAVING SUM(QTE) > 100);

4 rows affected.


[]

__2.3__ Compare the content of _table_ 'RBB' and the _view_ 'bons_buveurs' after the update. What differences do you see? Explain

We can see that the difference between the table RBB and the view 'bons-buveurs' return three tuples. The view is updated when the table CLIENT is updated. But one tuple that has been modifed was a transformation on the attribute TYPE 'petit' -> 'gros'. This tuple was not present at the creation of the view and was henc not added during the updating.

In [17]:
%%sql
SELECT bons_buveurs.*
FROM bons_buveurs 
    LEFT JOIN RBB ON (bons_buveurs.NB = RBB.NB)
WHERE RBB.NB IS NULL

Done.


NB,NOM,PRENOM,TYPE
44,Gide,Andre,gros
9,Ajar,Emile,gros
5,Audiberti,Jacques,gros


In [18]:
%%sql
SELECT COUNT(*) FROM RBB WHERE TYPE IN ('gros', 'moyen')

Done.


COUNT(*)
56


In [19]:
%%sql
SELECT COUNT(*) FROM CLIENT WHERE TYPE IN ('gros', 'moyen')

Done.


COUNT(*)
59


In [20]:
%%sql

SELECT *
FROM RBB
WHERE NB=2

Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen


__2.4__ Create a table 'RBA' with the same schema as 'buveurs_asec' which contains the tuples selected from 'buveurs_asec'

In [21]:
%%sql 
DROP TABLE IF EXISTS RBA;

CREATE TABLE RBA AS 
SELECT * 
FROM buveurs_asec;

Done.
Done.


[]

In [22]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__2.5__ Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to 'RBA'

In [23]:
%%sql 

INSERT INTO RBA 
VALUES (101, 'Loison', 'Olivier', 'moyen') ;

1 rows affected.


[]

In [24]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__2.6__ Compare the content of _table_ 'RBA' and the _view_ 'buveurs_asec'. What differences do you see? Explain

In [25]:
%%sql
SELECT RBA.*
FROM RBA
    LEFT JOIN buveurs_asec ON (RBA.NB = buveurs_asec.NB)
WHERE buveurs_asec.NB IS NULL

Done.


NB,NOM,PRENOM,TYPE
101,Loison,Olivier,moyen


___
# PART III: Design the database schema for posts in a Social Network

In this section you need to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.


__3.1__ Write and explain the design of the relations of your database

Answer: <br/>
- USER($\underline{email}$, Name, NickName, birth, IdAddress)
- ADDRESS($\underline{IdAddress}$, Street, city, State, Country, PostalCode)
- FRIENDS($\underline{IdFriends}$, email_1, email_2)
- POST($\underline{IdPost}$, email, IdFriends, Date, Text, Attachment, ConversationNb, Read)
<br/> <br/>
USER: In this social network, each user have a unic email. Name and Nickname can be redundant as people can be named identically. People of the same family or friends can share the same address. Hence, email is the unic key.
<br/> <br/>
ADDRESS: Several cities can have the same name of street; several state or country can own a same city name; each city can have several postal code. We are going to use an index IdAdress as a key of this relation.
<br/> <br/>
FRIENDS: A relationship between two users is determined by the ID of these 2 users. To avoid redundancy, email_1 always correspond to the first email in the alphanumeric order. We create an index IdFriends for easy use in the last relation POST.
<br/> <br/>
POST: To answer to the concept of "reply', we create an attribute ConversationNbr. This is an index of the conversation. Finally, the attribute 'Read' is equal to 1 if the post has been already read and 0 otherwise. 

__3.2__ Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

In [1]:
%% sql
DROP TABLE IF EXISTS userbox;

CREATE VIEW UserBox
SELECT U2.Nickname as NicknameF, Date, Text, Attachment
FROM USER U1, USER U2, FRIENDS F, POST P
WHERE U1.email = 'you@startup.com'
AND U1.email = F.email1 or U1.email F.email2 
AND P.IdFriends = F.Idfriends
GROUPBY ConversationNb
HAVING max(Date) < "CurrentDay - 10 days" or min(Read) = 0
ORDER BY LastPostDate DESC

-- Posts of a conversation wll be shown if its last post date is less than 10 days old or if one post has not been read
-- Conversation are sorted by their last post date 

UsageError: Cell magic `%%` not found.
